# Implementando um Sistema Especialista de Animais

Um exemplo do [Currículo AI para Iniciantes](http://github.com/microsoft/ai-for-beginners).

Neste exemplo, implementaremos um sistema baseado em conhecimento simples para determinar um animal com base em algumas características físicas. O sistema pode ser representado pela seguinte árvore AND-OR (esta é uma parte da árvore inteira, podemos facilmente adicionar mais algumas regras):

![](../../../../../../translated_images/pt-BR/AND-OR-Tree.5592d2c70187f283.webp)


## Nosso próprio shell de sistemas especialistas com inferência reversa

Vamos tentar definir uma linguagem simples para representação de conhecimento baseada em regras de produção. Usaremos classes do Python como palavras-chave para definir as regras. Basicamente, haverá 3 tipos de classes:
* `Ask` representa uma pergunta que precisa ser feita ao usuário. Ela contém o conjunto de respostas possíveis.
* `If` representa uma regra, e é apenas um açúcar sintático para armazenar o conteúdo da regra
* `AND`/`OR` são classes para representar ramos E/OU da árvore. Eles apenas armazenam a lista de argumentos dentro. Para simplificar o código, toda funcionalidade é definida na classe pai `Content`


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

Em nosso sistema, a memória de trabalho conteria a lista de **fatos** como **pares atributo-valor**. A base de conhecimento pode ser definida como um grande dicionário que associa ações (novos fatos que devem ser inseridos na memória de trabalho) a condições, expressas como expressões AND-OR. Além disso, alguns fatos podem ser `Perguntados`.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Para realizar a inferência reversa, definiremos a classe `Knowledgebase`. Ela conterá:
* `memory` de trabalho - um dicionário que mapeia atributos para valores
* `rules` do Knowledgebase no formato definido acima

Dois métodos principais são:
* `get` para obter o valor de um atributo, realizando a inferência se necessário. Por exemplo, `get('color')` obteria o valor de um slot de cor (ele perguntará se necessário e armazenará o valor para uso posterior na memória de trabalho). Se perguntarmos `get('color:blue')`, ele perguntará pela cor e então retornará o valor `y`/`n` dependendo da cor.
* `eval` realiza a inferência real, ou seja, percorre a árvore AND/OR, avalia sub-objetivos etc.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Agora vamos definir nossa base de conhecimento sobre animais e realizar a consulta. Note que esta chamada fará perguntas a você. Você pode responder digitando `y`/`n` para perguntas de sim ou não, ou especificando um número (0..N) para perguntas com respostas múltipla escolha mais longas.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Usando Experta para Inferência Direta

No próximo exemplo, tentaremos implementar inferência direta usando uma das bibliotecas para representação de conhecimento, [Experta](https://github.com/nilp0inter/experta). **Experta** é uma biblioteca para criação de sistemas de inferência direta em Python, que foi projetada para ser semelhante ao clássico sistema antigo [CLIPS](http://www.clipsrules.net/index.html).

Também poderíamos ter implementado encadeamento para frente nós mesmos sem muitos problemas, mas implementações ingênuas geralmente não são muito eficientes. Para uma correspondência de regras mais eficaz, um algoritmo especial [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) é usado.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

Nós definiremos nosso sistema como uma classe que herda de `KnowledgeEngine`. Cada regra é definida por uma função separada com a anotação `@Rule`, que especifica quando a regra deve ser acionada. Dentro da regra, podemos adicionar novos fatos usando a função `declare`, e a adição desses fatos resultará na chamada de mais regras pelo motor de inferência direta.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Uma vez que definimos uma base de conhecimento, populamos nossa memória de trabalho com alguns fatos iniciais e então chamamos o método `run()` para realizar a inferência. Você pode ver como resultado que novos fatos inferidos são adicionados à memória de trabalho, incluindo o fato final sobre o animal (se configurarmos todos os fatos iniciais corretamente).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução automática [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte oficial. Para informações críticas, recomenda-se a tradução profissional humana. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações equivocadas decorrentes do uso desta tradução.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
